<h2>Tree structure</h2>

In [1]:
"""
This is the tree structure. Each element of the tree is itself a tree.

The position_x and position_y are the absolute positions that the item 
will have on the html file.
The children vector is a vector of trees.
"""

class t_element:
    name = ""
    
    position_x = 0
    position_y = 0
    children = []
    
    def __init__(self, name_, children_):
        self.name = name_
        self.children = children_
    

<h2>Some functions to plot tree to html</h2>

In [2]:
## Element sizes in mm
default_element_height = 6
default_element_width  = 40
default_horizontal_clearence = 10
default_vertical_clearence = 2

def return_div(div_class, top, left, w, h, content):
    code = "<div class='" + div_class + "'  style='top:" + str(top) + "mm; left:" + str(left) + "mm;"
    code += " width:" + str(w) + "mm; height:" + str(h) + "mm; line-height:" + str(h) + "mm;'>"
    code += content
    code += "</div>\n"
    return code

def get_block_html(tree_element):
    top = tree_element.position_y * (default_element_height + default_vertical_clearence)
    left = tree_element.position_x * (default_element_width + default_horizontal_clearence)

    return return_div("tree_element",top,left,default_element_width,default_element_height,tree_element.name)

def get_line_html(tree_element_1, tree_element_2):
    ##
    ##  (x1,y1) ·------------· (x2,y2)
    ##                       |
    ##                       |
    ##                       |
    ##                       |
    ##               (x3,y3) ·-------------· (x4,y4)
    ##
    ##

    
    X1 = tree_element_1.position_x * (default_element_width + default_horizontal_clearence)
    Y1 = tree_element_1.position_y * (default_element_height + default_vertical_clearence)
    X2 = tree_element_2.position_x * (default_element_width + default_horizontal_clearence)
    Y2 = tree_element_2.position_y * (default_element_height + default_vertical_clearence)
    
    x1 = X1 + default_element_width
    y1 = Y1 + default_element_height * 0.5
    
    x2 = x1 + default_horizontal_clearence * 0.5
    y2 = y1
    
    x3 = x2
    y3 = Y2 + default_element_height * 0.5 
    
    x4 = X2
    y4 = y3
        
    #return_div(div_class, top, left, w, h, content)
    html = return_div("hor_line",y1,x1,x2-x1,1,"")
    
    if y3 > y2:
        html+= return_div("ver_line",y2,x2,1,y3-y2,"")
    else:
        html+= return_div("ver_line",y3,x2,1,y2-y3,"")
        
    html+= return_div("hor_line",y3,x3,x4-x3,1,"")
    
    return html


def get_whole_tree_html(tree, html_result):
    if len(tree.children) == 0:
        return get_block_html(tree)
        
    for child in tree.children:
        html_result += get_whole_tree_html(child,"")
        
        html_result += get_line_html(tree, child)
    
    return html_result + get_block_html(tree)


def write_file(tree_html_text):
    template = open("tree_template.html","r",encoding="UTF-8")
    template_text = template.read()
    template.close()
    
    output = open("tree.html","w+",encoding="UTF-8")
    output.write(template_text.replace("TREE_CONTENT",tree_html_text))
    output.close()


<h2>Calculate elements positions in a tree</h2>

In [3]:
def calc_elements_positions(tree, current_position_y, current_position_x):
    
    if len(tree.children) == 0:
        tree.position_y = current_position_y
        tree.position_x = current_position_x
        return current_position_y + 1
       
    tree.position_y = current_position_y
    for child in tree.children:
        current_position_y = calc_elements_positions(child, current_position_y, current_position_x + 1)
        
    tree.position_y = (tree.children[0].position_y + tree.children[-1].position_y)/2
    tree.position_x = current_position_x
    
    return current_position_y


<h2>Read gedcom file and create tree</h2>

In [22]:
gedfile = open("family_tree.ged","r",encoding="UTF-8")
ged = gedfile.read()
gedfile.close()

## Individuals

current_individual = ""
current_individual_birth = ""
current_individual_death = ""
individuals = {}

prev_line = ""

for line in ged.split("\n"):
    if line.startswith("0 @I"):
        current_individual = line.replace("0 @I","").split("@")[0]
  
    if line.startswith("1 NAME") and prev_line.startswith("0 @I"):
        individuals[current_individual] = t_element( line.replace("/","").replace("1 NAME ","") , [] )
    
    prev_line = line
        
## Families (relationships)

current_husband = ""
current_wife = ""
        
for line in ged.split("\n"):
    if line.startswith("1 HUSB"):
        current_husband = line.replace("1 HUSB ","").replace("@","").replace("I","")
    if line.startswith("1 WIFE"):
        current_wife = line.replace("1 WIFE ","").replace("@","").replace("I","")
    
    if line.startswith("1 CHIL"):
        child = line.replace("1 CHIL ","").replace("@","").replace("I","")
        individuals[current_husband].children.append(individuals[child])
        individuals[current_wife].children.append(individuals[child])   
        
print("Loaded " + str(len(individuals)) + " individuals")

Loaded 23 individuals


<hr>

<h2>Run</h2>

In [23]:
## define first member of the tree
first_member = individuals["19"]

## calculate elements positions
calc_elements_positions(first_member,0,0)

## get tree content
html_content = get_whole_tree_html(first_member, "")

## print to file
write_file(html_content)